In [1]:
import bs4
import requests
import json
import pandas as pd
import os

from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta

## NHS UK
https://digital.nhs.uk/data-and-information/publications/statistical/mi-potential-covid-19-symptoms-reported-through-nhs-pathways-and-111-online/latest


In [2]:
nhse = "https://digital.nhs.uk/data-and-information/publications/statistical/mi-potential-covid-19-symptoms-reported-through-nhs-pathways-and-111-online/latest"
NHSPathwaysCovid = "../src/data/NHSPathwaysCovid-latest.csv"

print('\n'+ NHSPathwaysCovid + '     ' + str(os.stat(NHSPathwaysCovid).st_size))
response = requests.get(nhse)
soup = BeautifulSoup(response.content)
blk=soup.find(class_="block-link")
pathways=blk.get('href')
print(pathways)

response = requests.get(pathways)
file = open(NHSPathwaysCovid, "wb")
file.write(response.content)
file.close
print('\n'+ NHSPathwaysCovid + '     ' + str(os.stat(NHSPathwaysCovid).st_size))


../src/data/NHSPathwaysCovid-latest.csv     6582729
https://files.digital.nhs.uk/22/A642D2/NHS%20Pathways%20Covid-19%20data%202020-07-05.csv

../src/data/NHSPathwaysCovid-latest.csv     6582729


## 111 Latest

In [3]:
One_Online = "../src/data/111_Online_latest.csv"

print('\n'+ One_Online + '     ' + str(os.stat(One_Online).st_size))
blk=soup.find_all(class_="block-link")
for url in blk:
    c = url.get('href')
    if '111' in c and 'mapped' not in c and 'Description' not in c and 'Meta' not in c:
        print(c)
        one=c
response = requests.get(one)
file = open(One_Online, "wb")
file.write(response.content)
file.close
print('\n'+ One_Online + '     ' + str(os.stat(One_Online).st_size))


../src/data/111_Online_latest.csv     4890152
https://files.digital.nhs.uk/9A/02BE37/111%20Online%20Covid-19%20data_2020-07-05.csv

../src/data/111_Online_latest.csv     4890152


## Testing Time Series

In [4]:
tests='https://www.gov.uk/guidance/coronavirus-covid-19-information-for-the-public'
tts_dl="../src/data/testing.csv"
response = requests.get(tests)
soup = BeautifulSoup(response.content)
blk=soup.find_all(class_="govuk-link")
for url in blk:
    link = url.get('href')
    if 'testing_time_series' in link:
        print(link)
        tts=link
response = requests.get(tts)
file = open(tts_dl, "wb")
file.write(response.content)
file.close

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/897770/2020-07-05_COVID-19_UK_testing_time_series.csv


<function BufferedWriter.close>

In [5]:
tests='https://www.gov.uk/guidance/coronavirus-covid-19-information-for-the-public'
tts_dl="../src/data/testing.csv"
print('\n'+ tts_dl + '     ' + str(os.stat(tts_dl).st_size))
response = requests.get(tests)
soup = BeautifulSoup(response.content)
blk=soup.find_all(class_="govuk-link")
for url in blk:
    c = url.get('href')
    if 'testing_time_series' in c:
        print(c)
        tts=c
response = requests.get(tts)
file = open(tts_dl, "wb")
file.write(response.content)
file.close
print('\n'+ tts_dl + '     ' + str(os.stat(tts_dl).st_size))


../src/data/testing.csv     81808
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/897770/2020-07-05_COVID-19_UK_testing_time_series.csv

../src/data/testing.csv     81808


## Cases Latest

In [6]:
cases = "https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv"
cdl = "../src/data/coronavirus-cases_latest.csv"

print(cdl + '     ' + str(os.stat(cdl).st_size))
response = requests.get(cases)
file = open(cdl, "wb")
file.write(response.content)
file.close
print(cdl + '     ' + str(os.stat(cdl).st_size))

../src/data/coronavirus-cases_latest.csv     3585022
../src/data/coronavirus-cases_latest.csv     3585022


## NHS England Deaths 2pm Weekdays
https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-daily-deaths/

In [8]:
if datetime.today().hour < 14:
    current = date.today() - timedelta(1)
    previous = date.today() - timedelta(2)
else:
    current = date.today() - timedelta(0)
    previous = date.today() - timedelta(1)
days = [current,previous]
i = 0
for d in days:
    nhse='https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/' + d.strftime('%m') + '/COVID-19-total-announced-deaths-' \
+ d.strftime('%-d-%B-%Y') + '.xlsx'
    nhse='https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/' + d.strftime('%m') + '/COVID-19-total-announced-deaths-' \
+ d.strftime('%-d-%b-%Y') + '.xlsx'
    print(nhse)
    df=pd.read_excel(nhse,skiprows=15,sheet_name='Tab4 Deaths by trust')
    df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
    df.drop(columns=['Awaiting verification','Total'], inplace=True)
    df.drop([0, 1],inplace=True)
    df.rename(columns={'NHS England Region': 'REGION', 'Code': 'REGION_CODE','Name': 'HOSPITAL', 'Up to 01-Mar-20': '2020-02-28',}, inplace=True)
    #DQ Issues
    #df.loc[(df['2020-02-28'] == 'RA4'),['2020-02-28']]='0'
    #df = df[df['REGION_CODE'].notna()]
    df.dropna(subset =['REGION_CODE'], inplace=True)

    df=pd.melt(df,id_vars=["REGION","REGION_CODE","HOSPITAL"], var_name="DATE", value_name="DEATHS")
    df['REGION'] = df['REGION'].map(lambda x: x.strip())
    df['DOWNLOAD_DATE'] = d.strftime('%Y-%m-%d')
    i = i + 1
    if i == 1:
        file = "../src/data/hospital_deaths_england_current.csv"
    elif i == 2:
        file = "../src/data/hospital_deaths_england_previous.csv"
    print(file + '     ' + str(os.stat(file).st_size))
    df.to_csv(file,index=False)
    print(file + '     ' + str(os.stat(file).st_size))

https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/07/COVID-19-total-announced-deaths-5-Jul-2020.xlsx


HTTPError: HTTP Error 404: Not Found

## ONS - Updated Weekly, Tuesdays 9:30am
https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/deathregistrationsandoccurrencesbylocalauthorityandhealthboard/2020

In [8]:
ons_page='https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/deathregistrationsandoccurrencesbylocalauthorityandhealthboard'
ons_file='/Users/i049374/OneDrive - SAP SE/Dev/Github/covid-nhs/db/src/data/ONS-Deaths-Latest.csv'
ons_dl = '/Users/i049374/OneDrive - SAP SE/Dev/Github/covid-nhs/db/src/data/ONS-tmp.xlsx'
print(ons_file + '     ' + str(os.stat(ons_file).st_size))

today=date.today() - timedelta(8)
week=today.strftime("%W")
ons = "https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fhealthandsocialcare%2fcausesofdeath%2fdatasets%2fdeathregistrationsandoccurrencesbylocalauthorityandhealthboard%2f2020/lahbtablesweek" + week + ".xlsx"

response = requests.get(ons)
file = open(ons_dl, "wb")
file.write(response.content)
file.close

df=pd.read_excel(ons_dl,skiprows=3,sheet_name='Occurrences - All data')
df.rename(columns={'Area name ': 'Area name'}, inplace=True)

df.to_csv(ons_file,index=False)
os.remove(ons_dl)

print(ons)
print(ons_file + '     ' + str(os.stat(ons_file).st_size))

/Users/i049374/OneDrive - SAP SE/Dev/Github/covid-nhs/db/src/data/ONS-Deaths-Latest.csv     6690109
https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fhealthandsocialcare%2fcausesofdeath%2fdatasets%2fdeathregistrationsandoccurrencesbylocalauthorityandhealthboard%2f2020/lahbtablesweek25.xlsx
/Users/i049374/OneDrive - SAP SE/Dev/Github/covid-nhs/db/src/data/ONS-Deaths-Latest.csv     6690109


## Done

## Pillar 2 Testing Data - weekly
#### Not needed, data now part of "cases" dataset
https://www.gov.uk/government/publications/national-covid-19-surveillance-reports

In [9]:
import re
pil2_web = 'https://www.gov.uk/government/publications/national-covid-19-surveillance-reports'
pil2_file = "../src/data/testing_Weekly_COVID19_report_data_w27.xlsx"

response = requests.get(pil2_web)
soup = BeautifulSoup(response.content)
blk=soup.find_all(href=re.compile("Weekly_COVID19_report_data"))
for url in blk:
    dl = url.get('href')
    print(dl)
    response = requests.get(dl)
    file = open(pil2_file, "wb")
    file.write(response.content)
    file.close


https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/897200/Weekly_COVID19_report_data_w27.xlsx
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/897200/Weekly_COVID19_report_data_w27.xlsx
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/896777/Weekly_COVID19_report_data_current.xlsx
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/896777/Weekly_COVID19_report_data_current.xlsx


In [10]:
df=pd.read_excel(pil2_file,skiprows=7,sheet_name='Figure 9. Weekly rates UTLA')

In [11]:
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
df.head(5)

,UTLA code,UTLA name,"Rate per 100,000 population"
0,E09000002,Barking and Dagenham,1.42
1,E09000003,Barnet,2.30
2,E08000016,Barnsley,54.65
3,E06000022,Bath and North East Somerset,1.04
4,E06000055,Bedford,41.95
